In [1]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [3]:
from skimage.feature import hog

train_feats=[]
test_feats=[]
train_labs=[]
test_labs=[]

for i,img in enumerate(x_train):
  fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
  train_feats.append(fd)
  train_labs.append(y_train[i])

train_feats=np.array(train_feats)
train_labs=np.array(train_labs)
train_feats.shape

(50000, 324)

In [4]:
for i,img in enumerate(x_test):
  fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
  test_feats.append(fd)
  test_labs.append(y_test[i])

test_feats=np.array(test_feats)
test_labs=np.array(test_labs)
test_feats.shape

(10000, 324)

In [5]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=42, max_iter=1000, algorithm='elkan').fit(train_feats)

preds=kmeans.predict(test_feats) # use features extracted from denseNet
preds.shape

(10000,)

In [8]:
kmeans.labels_

array([5, 7, 9, ..., 1, 6, 9], dtype=int32)

In [14]:
y_test[:5]

array([[3],
       [8],
       [8],
       [0],
       [6]], dtype=uint8)

In [15]:
y1=np.array([val[0] for val in y_test])
y1.shape

(10000,)

In [9]:
preds[:3]

array([2, 6, 8], dtype=int32)

In [10]:
Counter(preds)

Counter({2: 1297,
         6: 836,
         8: 915,
         5: 1097,
         7: 1035,
         4: 793,
         0: 867,
         1: 940,
         3: 1350,
         9: 870})

In [13]:
y1[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
# assign predicted clusters to actual classes to get the accuracy
from collections import Counter

correct=0
mapping={}

for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices for a particular class
  p=preds[inds] # predictions made for that class
  y2=y1[inds]
  counts=dict(Counter(p))

  print("y2: ", y2[:2])
  print("counts: ", counts)
  print("----------------")

y2:  [0 0]
counts:  {6: 289, 1: 103, 8: 171, 4: 72, 0: 65, 9: 166, 7: 61, 3: 22, 5: 23, 2: 28}
----------------
y2:  [1 1]
counts:  {4: 39, 5: 74, 9: 255, 6: 136, 7: 121, 8: 254, 0: 26, 2: 82, 1: 10, 3: 3}
----------------
y2:  [2 2]
counts:  {3: 184, 0: 187, 2: 81, 8: 73, 4: 190, 9: 58, 5: 66, 1: 109, 7: 37, 6: 15}
----------------
y2:  [3 3]
counts:  {2: 176, 0: 98, 3: 310, 7: 69, 5: 128, 4: 75, 1: 72, 9: 41, 8: 24, 6: 7}
----------------
y2:  [4 4]
counts:  {3: 155, 7: 58, 5: 181, 1: 160, 4: 66, 8: 51, 2: 187, 0: 80, 9: 50, 6: 12}
----------------
y2:  [5 5]
counts:  {5: 117, 4: 61, 3: 407, 9: 15, 7: 41, 2: 151, 0: 91, 1: 104, 6: 2, 8: 11}
----------------
y2:  [6 6]
counts:  {5: 164, 7: 26, 0: 205, 2: 211, 4: 180, 8: 49, 9: 45, 3: 63, 1: 55, 6: 2}
----------------
y2:  [7 7]
counts:  {3: 150, 2: 217, 5: 230, 8: 28, 1: 165, 9: 21, 0: 64, 4: 60, 6: 6, 7: 59}
----------------
y2:  [8 8]
counts:  {6: 301, 8: 164, 0: 21, 9: 155, 7: 234, 1: 52, 3: 11, 4: 29, 5: 15, 2: 18}
---------------

In [17]:
#HOG + clustering
mapping={
    0:6,
    1:9,
    2:4,
    3:3,
    4:2,
    5:3,
    6:0,
    7:5,
    8:6,
    9:7
}
correct=0
for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices
  p=preds[inds]
  y2=y1[inds]
  counts=dict(Counter(p))
  correct+=counts[mapping[y2[0]]]
print("accuracy: ", correct*100/len(y1))

accuracy:  27.03
